In [ ]:
full = df.merge(df[rolling_cols + ["next_opp", "next_date", "teamname"]], left_on=["teamname", "next_date"], \
                right_on=["next_opp", "next_date"])

In [ ]:
from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier

rr = RidgeClassifier(alpha=1)

rf = RandomForestClassifier(max_depth=3, max_samples = .4, n_estimators = 100)

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, 
                                n_features_to_select=20, 
                                direction="forward",
                                cv=split,
                                n_jobs=1
                               )

In [ ]:
def near_split(x, num_bins): #Split my df into equal splits to perform backtesting
    quotient, remainder = divmod(x, num_bins)
    bins = [quotient + 1] * remainder + [quotient] * (num_bins - remainder)
    count = 0
    new_list = []
    for b in bins:
        count += b
        new_list.append(count)
    return new_list

splits = near_split(df.shape[0],5)

In [ ]:
def backtest(data, model, predictors, target, start=0, step=1):
    all_predictions = []

    for i in range(0,len(splits)-1):
        train = data.loc[:splits[i],:]
        test = data.loc[:splits[i]+2568,:]
        
        model.fit(train[predictors], train[target])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test[target], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [ ]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [ ]:
sfs.fit(full[selected_columns], full["target"])

In [ ]:
predictors = list(selected_columns[sfs.get_support()])

In [ ]:
predictions = backtest(full, rr, predictors,'target')

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(predictions.actual,predictions.prediction)